In [4]:
# load libraries
import pandas as pd
import os
import numpy as np
import datetime
from torch.utils.data import Dataset, DataLoader
from torch import save

In [5]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

# change me, have to point to the Meteotrentino drive folder
wk =  "/content/drive/My Drive/10. Meteotrentino/"

# folder for the csv
subfolder = "CSVs/joinPioggia"


data_path = os.path.join(wk, subfolder)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
files = os.listdir(data_path)
print(files)
df_list = []
for f in files:
  f1 = pd.read_csv(
      os.path.join(data_path, f),
      dtype = {
          'label' : str, 
          'date': str, 
          'value': float, 
          'validation_code' : int,
          'datetime' : str} 
      
      )
  
  f1['datetime'] = pd.to_datetime(
      f1['datetime'],
      format = '%Y-%m-%d %H:%M:%S'
      )
  f1.drop(f1.columns[0], axis = 1, inplace = True)
  df_list.append(f1)

['stationT0129.csv', 'stationT0135.csv', 'stationT0101.csv', 'stationT0408.csv']


In [7]:
df_total = pd.concat(df_list)
avaibile_stations = df_total.label.unique()
print("Avaibile stations = "+ str(avaibile_stations))


Avaibile stations = ['T0129' 'T0135' 'T0101' 'T0408']


In [8]:
# check if total lenght is ok
print(df_total.reset_index())
suml = []
for d in df_list:
  suml.append((len(d)))
print(sum(suml))

          index  label        date   time  value  validation_code  \
0             0  T0129  01/01/2014  00:00    0.0                1   
1             1  T0129  01/01/2014  00:05    0.0                1   
2             2  T0129  01/01/2014  00:10    0.0                1   
3             3  T0129  01/01/2014  00:15    0.0                1   
4             4  T0129  01/01/2014  00:20    0.0                1   
...         ...    ...         ...    ...    ...              ...   
3364075  841125  T0408  31/12/2021  23:35    0.0                1   
3364076  841126  T0408  31/12/2021  23:40    0.0                1   
3364077  841127  T0408  31/12/2021  23:45    0.0                1   
3364078  841128  T0408  31/12/2021  23:50    0.0                1   
3364079  841129  T0408  31/12/2021  23:55    0.0                1   

                   datetime  
0       2014-01-01 00:00:00  
1       2014-01-01 00:05:00  
2       2014-01-01 00:10:00  
3       2014-01-01 00:15:00  
4       2014-01-01 00

In [9]:
#check uniques dates:
date_unique = pd.unique(df_total['datetime'])
print(len(date_unique))

841501


In [10]:
# gorup by the dates
df_total_gb = df_total.pivot(index = 'datetime', columns='label', values = ['value', 'time', 'date', 'validation_code'])

df_total_gb

value                     time                       \
label               T0101 T0129 T0135 T0408  T0101  T0129  T0135  T0408   
datetime                                                                  
2014-01-01 00:00:00   0.0   0.0   0.0   0.0  00:00  00:00  00:00  00:00   
2014-01-01 00:05:00   0.0   0.0   0.0   0.0  00:05  00:05  00:05  00:05   
2014-01-01 00:10:00   0.0   0.0   0.0   0.0  00:10  00:10  00:10  00:10   
2014-01-01 00:15:00   0.0   0.0   0.0   0.0  00:15  00:15  00:15  00:15   
2014-01-01 00:20:00   0.0   0.0   0.0   0.0  00:20  00:20  00:20  00:20   
...                   ...   ...   ...   ...    ...    ...    ...    ...   
2021-12-31 23:35:00   0.0   0.0   0.0   0.0  23:35  23:35  23:35  23:35   
2021-12-31 23:40:00   0.0   0.0   0.0   0.0  23:40  23:40  23:40  23:40   
2021-12-31 23:45:00   0.0   0.0   0.0   0.0  23:45  23:45  23:45  23:45   
2021-12-31 23:50:00   0.0   0.0   0.0   0.0  23:50  23:50  23:50  23:50   
2021-12-31 23:55:00   0.0   0.0   0.0   0.0  23:55  23:55  23:55  23:55   

                           date                                      \
label                     T0101       T0129       T0135       T0408   
datetime                                                              
2014-01-01 00:00:00  01/01/2014  01/01/2014  01/01/2014  01/01/2014   
2014-01-01 00:05:00  01/01/2014  01/01/2014  01/01/2014  01/01/2014   
2014-01-01 00:10:00  01/01/2014  01/01/2014  01/01/2014  01/01/2014   
2014-01-01 00:15:00  01/01/2014  01/01/2014  01/01/2014  01/01/2014   
2014-01-01 00:20:00  01/01/2014  01/01/2014  01/01/2014  01/01/2014   
...                         ...         ...         ...         ...   
2021-12-31 23:35:00  31/12/2021  31/12/2021  31/12/2021  31/12/2021   
2021-12-31 23:40:00  31/12/2021  31/12/2021  31/12/2021  31/12/2021   
2021-12-31 23:45:00  31/12/2021  31/12/2021  31/12/2021  31/12/2021   
2021-12-31 23:50:00  31/12/2021  31/12/2021  31/12/2021  31/12/2021   
2021-12-31 23:55:00  31/12/2021  31/12/2021  31/12/2021  31/12/2021   

                    validation_code                    
label                         T0101 T0129 T0135 T0408  
datetime                                               
2014-01-01 00:00:00               1     1     1     1  
2014-01-01 00:05:00               1     1     1     1  
2014-01-01 00:10:00               1     1     1     1  
2014-01-01 00:15:00               1     1     1     1  
2014-01-01 00:20:00               1     1     1     1  
...                             ...   ...   ...   ...  
2021-12-31 23:35:00               1     1     1     1  
2021-12-31 23:40:00               1     1     1     1  
2021-12-31 23:45:00               1     1     1     1  
2021-12-31 23:50:00               1     1     1     1  
2021-12-31 23:55:00               1     1     1     1  

[841501 rows x 16 columns]

In [11]:
import matplotlib.pyplot as plt

In [12]:
for station in  avaibile_stations:
  unique, counts = np.unique(df_total_gb['validation_code'][station].values, return_counts=True)

  print("Sensor " +str(station))
  print("Good values: "+str(unique[0])+" counts = "+str(counts[0]))
  # plot to check
  #plt.plot(unique, counts, marker = 'x', linestyle = "None")
  #plt.title(station)
  #plt.show()

Sensor T0129
Good values: 1 counts = 894
Sensor T0135
Good values: 1 counts = 3133
Sensor T0101
Good values: 1 counts = 1762
Sensor T0408
Good values: 1 counts = 3817


In [13]:
# drop T0129, the worst one and hope 
del df_list[0] 
avaibile_stations = avaibile_stations[1:]

In [14]:

print(avaibile_stations)

['T0135' 'T0101' 'T0408']


In [15]:
new_df_list = []
for df in df_list:
  sensor_name = df['label'].values[0]
  df_tmp = df.set_index(['datetime'])
  df_tmp.drop(['date','time','label'],axis=1,inplace=True)
  df_tmp.rename(columns={'value':f'value_{sensor_name}','validation_code':f'validation_code_{sensor_name}'},inplace=True)
  new_df_list.append(df_tmp)

In [16]:
!git clone https://github.com/eliazonta/AI-Challenge-HIT 

Cloning into 'AI-Challenge-HIT'...
remote: Enumerating objects: 451, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 451 (delta 81), reused 90 (delta 37), pack-reused 269
Receiving objects: 100% (451/451), 24.64 MiB | 14.22 MiB/s, done.
Resolving deltas: 100% (196/196), done.


In [17]:
import sys
sys.path.append("/content/AI-Challenge-HIT/python_code/notebooks/")
sys.path.append("/content/AI-Challenge-HIT/python_code/notebooks")
sys.path.append("/content/AI-Challenge-HIT/python_code/notebooks/TSF")
sys.path.append("/content/AI-Challenge-HIT/python_code/notebooks/TSF/")

In [19]:


from importlib import reload  # Python 3.4+
import data_splitter
data_splitter = reload(data_splitter)
from data_splitter import *
df_multiple = pd.concat(new_df_list,axis=1)
df_multiple['datetime'] = df_multiple.index

In [20]:
df_train_origin,df_prediction_origin,n_previsions = split_multiple_df_for_TSF(
    df_multiple,
    avaibile_stations,
    24,1,
    500,
    log_file='/tmp/schifo'
    )

Logging to /tmp/schifo
index = 500	 current date = 2014-01-28 02:00:00	 to the end date = 2021-12-31 23:55:00
index = 1000	 current date = 2014-02-19 18:00:00	 to the end date = 2021-12-31 23:55:00
index = 1500	 current date = 2014-03-17 14:00:00	 to the end date = 2021-12-31 23:55:00
index = 2000	 current date = 2014-04-12 11:00:00	 to the end date = 2021-12-31 23:55:00
index = 2500	 current date = 2014-05-10 14:00:00	 to the end date = 2021-12-31 23:55:00
index = 3000	 current date = 2014-06-04 10:00:00	 to the end date = 2021-12-31 23:55:00
index = 3500	 current date = 2014-06-28 06:00:00	 to the end date = 2021-12-31 23:55:00
index = 4000	 current date = 2014-07-21 02:00:00	 to the end date = 2021-12-31 23:55:00
index = 4500	 current date = 2014-08-10 22:00:00	 to the end date = 2021-12-31 23:55:00
index = 5000	 current date = 2014-09-01 18:00:00	 to the end date = 2021-12-31 23:55:00
index = 5500	 current date = 2014-09-26 14:00:00	 to the end date = 2021-12-31 23:55:00
index = 60

In [ ]:
# Save FIXME: no CSV format
# df_train_origin.to_csv('/content/drive/MyDrive/pioggia_join_train.csv')
# df_prediction_origin.to_csv('/content/drive/MyDrive/pioggia_join_prediction.csv')

In [14]:
# load 
# df_train_origin = pd.read_csv('/content/drive/MyDrive/pioggia_join_train.csv')
# df_prediction_origin = pd.read_csv('/content/drive/MyDrive/pioggia_join_prediction.csv')


In [45]:

df_train = df_train_origin
df_test = df_prediction_origin

In [46]:
# if exists, remove the 12th column
print(df_test.columns)
df_test = df_test.drop([12, 'val_12'], axis = 1)
print(df_test.columns)
print(df_test)

Index(['sensor_code', 'in_datetime',             0,             1,
                   2,             3,             4,             5,
                   6,             7,             8,             9,
                  10,            11,       'val_0',       'val_1',
             'val_2',       'val_3',       'val_4',       'val_5',
             'val_6',       'val_7',       'val_8',       'val_9',
            'val_10',      'val_11',            12,      'val_12'],
      dtype='object')
Index(['sensor_code', 'in_datetime',             0,             1,
                   2,             3,             4,             5,
                   6,             7,             8,             9,
                  10,            11,       'val_0',       'val_1',
             'val_2',       'val_3',       'val_4',       'val_5',
             'val_6',       'val_7',       'val_8',       'val_9',
            'val_10',      'val_11'],
      dtype='object')
                 sensor_code         in_dateti

In [47]:
# remove data with NaN in training
bad_rows = df_train.isnull().values.any(axis=1)
df_train = df_train.iloc[~bad_rows]
df_test = df_test.iloc[~bad_rows]

bad_rows = df_test.isnull().values.any(axis=1)
df_train = df_train.iloc[~bad_rows].reset_index(drop=True)
df_test = df_test.iloc[~bad_rows].reset_index(drop=True)



In [48]:
print(df_train)
print(df_test)

                 sensor_code         in_datetime                0  \
0      [T0135, T0101, T0408] 2014-01-02 00:05:00  [0.0, 0.0, 0.0]   
1      [T0135, T0101, T0408] 2014-01-02 01:05:00  [0.0, 0.0, 0.0]   
2      [T0135, T0101, T0408] 2014-01-02 02:05:00  [0.0, 0.0, 0.0]   
3      [T0135, T0101, T0408] 2014-01-02 03:05:00  [0.0, 0.0, 0.0]   
4      [T0135, T0101, T0408] 2014-01-02 04:05:00  [0.0, 0.0, 0.0]   
...                      ...                 ...              ...   
55605  [T0135, T0101, T0408] 2021-12-30 18:05:00  [0.0, 0.0, 0.0]   
55606  [T0135, T0101, T0408] 2021-12-30 19:05:00  [0.0, 0.0, 0.0]   
55607  [T0135, T0101, T0408] 2021-12-30 20:05:00  [0.0, 0.0, 0.0]   
55608  [T0135, T0101, T0408] 2021-12-30 21:05:00  [0.0, 0.0, 0.0]   
55609  [T0135, T0101, T0408] 2021-12-30 22:05:00  [0.0, 0.0, 0.0]   

                     1                2                3                4  \
0      [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]   
1      [0.0, 0.0,

In [49]:
from torch.autograd import anomaly_mode
# all rows with anomalies
list_validation_columns = [f"val_{idx}" for idx in range(n_previsions)]
validations = df_test[list_validation_columns]
#print(validations)

mask = []
drop_mask = []
# cycle over the rows (day)
for v in validations.iterrows():
  # dirty trick beacause i get a tuple. I need the array of validations
  v = v[1]
  anomaly_detected = 0
  drop = 0
  # print(v)
  # cycle over the validation of that day (n_previsions)
  for k in v:
    #print(k)
    # if  (k != [1.0, 1.0, 1.0]):
    #   # got an anomaly on the row, it's enough to stop the process
    #   anomaly_detected = 1
    #   break
    if (np.isnan(k).any()):
       drop = 1
       break
    
    # if(drop == 1 or anomaly_detected == 1):
    #   break
  if(drop == 1):
    drop_mask.append(True)
  else:
    drop_mask.append(False)

  
#mask = np.array(mask)

#print(validations)

#dataset = {}
#
## split data in training and test (to check generalization)
#
## split data in training and test (to check generalization)
## training set
#training_features = df_train_without_anomalies.sample(frac=0.8)
#training_target = df_prediction_without_anomalies.iloc[training_features.index.values]
## test set
#test_features = df_train_without_anomalies.drop(training_features.index)
#test_target = df_prediction_without_anomalies.iloc[test_features.index]
## add anomalies to test set
#test_features = pd.concat([test_features,features_anomalies],axis = 0)
#test_target = pd.concat([test_target,target_anomalies],axis = 0)
#
#training_features = training_features.reset_index(drop=True)
#training_target = training_target.reset_index(drop=True)
#test_features = test_features.reset_index(drop=True)
#test_target = test_target.reset_index(drop=True)


In [50]:

print(len(drop_mask))
n,c = np.unique(drop_mask, return_counts = True)
print(n)
print(c)
print(df_test[drop_mask])

55610
[False  True]
[55159   451]
                 sensor_code         in_datetime                0  \
50     [T0135, T0101, T0408] 2014-01-04 02:05:00  [0.2, 0.4, 0.4]   
202    [T0135, T0101, T0408] 2014-01-11 10:05:00  [0.0, 0.0, 0.0]   
228    [T0135, T0101, T0408] 2014-01-13 19:05:00  [0.0, 0.0, 0.0]   
372    [T0135, T0101, T0408] 2014-01-20 19:05:00  [0.0, 0.0, 0.0]   
395    [T0135, T0101, T0408] 2014-01-22 18:05:00  [0.0, nan, 0.0]   
...                      ...                 ...              ...   
50150  [T0135, T0101, T0408] 2021-05-12 10:05:00  [0.0, 0.0, 0.0]   
52432  [T0135, T0101, T0408] 2021-08-16 13:05:00  [0.0, 0.0, 0.0]   
53580  [T0135, T0101, T0408] 2021-10-04 09:05:00  [0.0, 0.0, 0.0]   
54733  [T0135, T0101, T0408] 2021-11-22 10:05:00  [0.0, nan, 0.0]   
55409  [T0135, T0101, T0408] 2021-12-21 14:05:00  [nan, 0.0, 0.0]   

                     1                2                3                4  \
50     [0.0, 0.0, 0.2]  [0.2, 0.4, 0.2]  [nan, 0.4, 0.2]  [n

In [51]:
# cavo i nan
print(df_train[~(np.array(drop_mask))])
df_train = df_train[~(np.array(drop_mask))]
df_test = df_test[~(np.array(drop_mask))]

                 sensor_code         in_datetime                0  \
0      [T0135, T0101, T0408] 2014-01-02 00:05:00  [0.0, 0.0, 0.0]   
1      [T0135, T0101, T0408] 2014-01-02 01:05:00  [0.0, 0.0, 0.0]   
2      [T0135, T0101, T0408] 2014-01-02 02:05:00  [0.0, 0.0, 0.0]   
3      [T0135, T0101, T0408] 2014-01-02 03:05:00  [0.0, 0.0, 0.0]   
4      [T0135, T0101, T0408] 2014-01-02 04:05:00  [0.0, 0.0, 0.0]   
...                      ...                 ...              ...   
55605  [T0135, T0101, T0408] 2021-12-30 18:05:00  [0.0, 0.0, 0.0]   
55606  [T0135, T0101, T0408] 2021-12-30 19:05:00  [0.0, 0.0, 0.0]   
55607  [T0135, T0101, T0408] 2021-12-30 20:05:00  [0.0, 0.0, 0.0]   
55608  [T0135, T0101, T0408] 2021-12-30 21:05:00  [0.0, 0.0, 0.0]   
55609  [T0135, T0101, T0408] 2021-12-30 22:05:00  [0.0, 0.0, 0.0]   

                     1                2                3                4  \
0      [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]   
1      [0.0, 0.0,

In [52]:
#reset index, and redo for anomalies
df_train = df_train.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [53]:
# look for anomalies
list_validation_columns = [f"val_{idx}" for idx in range(n_previsions)]
validations = df_test[list_validation_columns]

mask = []
# cycle over the rows (day)
for v in validations.iterrows():
  # dirty trick beacause i get a tuple. I need the array of validations
  v = v[1]
  anomaly_detected = 0
  drop = 0
  # print(v)
  # cycle over the validation of that day (n_previsions)
  for k in v:
    #print(k)
    if  (k != [1.0, 1.0, 1.0]):
      # got an anomaly on the row, it's enough to stop the process
      anomaly_detected = 1
      break
    
    # if(drop == 1 or anomaly_detected == 1):
    #   break
  if(anomaly_detected == 1):
    mask.append(True)
  else:
    mask.append(False)


In [54]:
print(len(mask))
n,c = np.unique(mask, return_counts = True)
print(n)
print(c)
print(df_test[mask])


55159
[False  True]
[55151     8]
                 sensor_code         in_datetime                0  \
6049   [T0135, T0101, T0408] 2014-10-20 23:05:00  [0.0, 0.0, 0.0]   
6573   [T0135, T0101, T0408] 2014-11-14 02:05:00  [0.0, 0.0, 0.0]   
14549  [T0135, T0101, T0408] 2015-10-27 04:05:00  [0.0, 0.0, 0.0]   
17586  [T0135, T0101, T0408] 2016-03-05 00:05:00  [0.2, 0.2, 0.2]   
20332  [T0135, T0101, T0408] 2016-08-09 00:05:00  [0.0, 0.0, 0.0]   
27005  [T0135, T0101, T0408] 2017-11-24 10:05:00  [0.0, 0.2, 0.0]   
40469  [T0135, T0101, T0408] 2020-03-23 00:05:00  [0.0, 0.0, 0.2]   
40602  [T0135, T0101, T0408] 2020-03-31 01:05:00  [0.0, 0.0, 0.0]   

                     1                2                3                4  \
6049   [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]   
6573   [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]   
14549  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]   
17586  [0.2, 0.0, 0.2]  [0.0, 0.2, 0

In [55]:
index_anomalies = df_test[mask].index
print(index_anomalies)

Int64Index([6049, 6573, 14549, 17586, 20332, 27005, 40469, 40602], dtype='int64')


In [56]:
# df with anomalies only
features_anomalies = df_train.iloc[index_anomalies]
target_anomalies = df_test.iloc[index_anomalies]

print(features_anomalies)
print(target_anomalies)

                 sensor_code         in_datetime                0  \
6049   [T0135, T0101, T0408] 2014-10-20 23:05:00  [0.0, 0.0, 0.0]   
6573   [T0135, T0101, T0408] 2014-11-14 02:05:00  [0.0, 0.0, 0.0]   
14549  [T0135, T0101, T0408] 2015-10-27 04:05:00  [0.0, 0.0, 0.0]   
17586  [T0135, T0101, T0408] 2016-03-05 00:05:00  [0.0, 0.0, 0.0]   
20332  [T0135, T0101, T0408] 2016-08-09 00:05:00  [0.0, 0.0, 0.0]   
27005  [T0135, T0101, T0408] 2017-11-24 10:05:00  [0.0, 0.0, 0.0]   
40469  [T0135, T0101, T0408] 2020-03-23 00:05:00  [0.0, 0.0, 0.0]   
40602  [T0135, T0101, T0408] 2020-03-31 01:05:00  [0.0, 0.0, 0.0]   

                     1                2                3                4  \
6049   [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]   
6573   [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]   
14549  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]   
17586  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0

In [57]:

## remove those rows from the whole dataset
df_train_without_anomalies = df_train.drop(index_anomalies).reset_index(drop=True)
df_prediction_without_anomalies = df_test.drop(index_anomalies).reset_index(drop=True)

print(len(df_train_without_anomalies.index))
print(len(df_train.index))

print(len(df_prediction_without_anomalies.index))
print(len(df_test.index))
#print(features_anomalies['in_datetime'])
#### DA METTERE LO UNIQUE QUI!!!!
#
##df_prediction.to_csv( wk + csv_subfolder + "/" + station + "_" + selected_dataset + "_" + "prediction.csv") #Save it on GDrive


55151
55159
55151
55159


In [58]:
splits = ['train','test']
shuffle = {'train':True,'test':False}
dataset = {}

In [59]:

# split data in training and test (to check generalization)

# split data in training and test (to check generalization)
# training set
training_features = df_train_without_anomalies.sample(frac=0.8)
training_target = df_prediction_without_anomalies.iloc[training_features.index.values]
# test set
test_features = df_train_without_anomalies.drop(training_features.index)
test_target = df_prediction_without_anomalies.iloc[test_features.index]
# add anomalies to test set
test_features = pd.concat([test_features,features_anomalies],axis = 0)
test_target = pd.concat([test_target,target_anomalies],axis = 0)

training_features = training_features.reset_index(drop=True)
training_target = training_target.reset_index(drop=True)
test_features = test_features.reset_index(drop=True)
test_target = test_target.reset_index(drop=True)


In [60]:
import torch
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
  def __init__(self, df_train, df_test):
    features = df_train.drop(['sensor_code','in_datetime'],axis=1)
    target = df_test.drop(['sensor_code','in_datetime']+[f"val_{idx}" for idx in range(n_previsions)],axis=1)
    self.features = torch.FloatTensor(np.array([list(features[i].values) for i in range(len(features.columns))]))
    self.target = torch.FloatTensor(np.array([list(target[i].values) for i in range(len(target.columns))]))
    mean = self.features.mean()
    std = self.features.std()
    # self.features = (self.features-mean)/std
    # self.target = (self.target-mean)/std
    self.sensor_code = np.array(df_train['sensor_code'].values) 
    self.in_datetime = df_train['in_datetime'].values
    labels = df_test[[f"val_{idx}" for idx in range(n_previsions)]]
    self.labels = torch.FloatTensor(np.array([list(labels[f"val_{i}"].values) for i in range(len(labels.columns))]))
    # to be explained
    self.features = self.features.permute((1,0,2))
    self.target = self.target.permute((1,0,2))
    self.labels = self.labels.permute((1,0,2))

  def __len__(self):
    return len(self.sensor_code)

  def __getitem__(self, idx):
    sensor_code = self.sensor_code[idx]
    date = self.in_datetime[idx]
    features = self.features[idx]
    labels = self.labels[idx]
    target = self.target[idx]
    return {"sensor_code": sensor_code, "date":date,"features": features, "target":target,"labels":labels}

In [61]:
custom_1 = CustomDataset(df_train, df_test)

In [62]:

dataset['train'] = CustomDataset(training_features, training_target)
dataset['test'] = CustomDataset(test_features, test_target)
save(dataset['train'], os.path.join(wk, 'DatasetPytorch', 'pioggia', "train_join_tsf.pt")) #Save it on GDrive
save(dataset['test'], os.path.join(wk, 'DatasetPytorch', 'pioggia', "test_join_tsf.pt")) #Save it on GDrive
print (station + " saved")



T0408 saved


In [ ]:

dataset['train'] = CustomDataset(training_features, training_target)
dataset['test'] = CustomDataset(test_features, test_target)
save(dataset['train'], pt_subfolder + station + "_train_"+ selected_dataset + "_tsf.pt") #Save it on GDrive
save(dataset['test'],  pt_subfolder + station + "_test_"+ selected_dataset + "_tsf.pt")
print (station + " saved")